In [1]:
from test_utils import *
from swayam import *

### Tool Calling with Swayam Tools

Let's now start to link the functionality of an LLM Agent with a locally available function calls.

In [2]:
router = Router(run_id="test")

#### Let's see what happens without a function call

In [3]:
router.execute(Prompt.user_prompt("What's the weather like in Boston?"))

Executing Conversation with 1 step(s)


ChatCompletionMessage(content="I don't have real-time data access to provide current weather conditions. However, you can easily check the weather in Boston by using a weather website, a weather app, or a search engine. Typically, Boston experiences cold winters, warm summers, and variable weather in spring and fall. If you need information about typical weather patterns for a specific time of year, feel free to ask!", refusal=None, role='assistant', function_call=None, tool_calls=None)

### Some Dummy Functions

Let's create some dummy functions

In [4]:
from enum import Enum
class UnitEnum(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"

import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit):
    import random
    i = random.randint(-10, 40)
    if unit.lower() == "celsius":
        return f"{i}°C"
    elif unit.lower() == "fahrenheit":
        return f"{i*9/5 + 32}°F"
    else:
        raise ValueError("Invalid unit")
    
def calculate(expression):
    return eval(expression)

def search_email(text):
    import re
    emails = re.findall(r'([\w\.-]+@[\w\.-]+)', text)
    if emails:
        return emails
    else:
        return None

## Tool Creation

As a part of an LLM workflow, we want to trigger the above local function based on response from the LLM (using function calling).

Before we can do that, we need to create a sort of bridge between a function from the view point of an LLM vs what it means in Python.

Here we see how we can create convert a function into a Tool.

### ToolBuilder

We use the tool builder to wrap the function created above.
1. Provide the function callable and describe it.
2. Add all arguments, while describing their types and purposes.

In [5]:
from swayam import Tool

def create_get_weather_info():
    builder = Tool.builder("WeatherInfo", target=get_current_weather, desc="Get the current weather in a given location")
    builder.add_field("location", type=str, desc="Location to get weather information for")
    builder.add_field("unit", type=UnitEnum, desc="Unit of temperature", default=UnitEnum.CELSIUS)
    return builder.build()

def create_calc_tool():
    builder = Tool.builder("Calculator", target=calculate, desc="Evaluates a mathematical expression")
    builder.add_field("expression", type=str, desc="The expression to be evaluated.")
    return builder.build()

def create_search_email_tool():
    builder = Tool.builder("EmailSearch", target=search_email, desc="Searches email addresses in a given text")
    builder.add_field("text", type=str, desc="The target text in which emails are to be searched.")
    return builder.build()

WeatherInfo = create_get_weather_info()
Calculator = create_calc_tool()
EmailSearch = create_search_email_tool()

### Tool Execution

The key thing to keep in mind is that all arguments are passed as keyword arguments irrespective of the underlying function.

In [6]:
WeatherInfo(location="dummy")

ToolResponse: 33°C

In [7]:
WeatherInfo(**{"location": "dummy"})

ToolResponse: 19°C

In [8]:
WeatherInfo(**{"location": "dummy", "unit":"fahrenheit"})

ToolResponse: 48.2°F

In [9]:
Calculator(expression="2 + 5")

ToolResponse: 7

In [10]:
EmailSearch(text="This is a sample text with a@b.com address")

ToolResponse: ['a@b.com']

In [11]:
EmailSearch(text="This is a sample text with no email address")

ToolResponse: None

### Tool Definition

One of the key benefits of creating a Tool is that it self describes itself, in a way compatible with an LLM.

In [12]:
print(WeatherInfo.definition)

{'type': 'function', 'function': {'name': 'WeatherInfo', 'description': 'Get the current weather in a given location', 'parameters': {'properties': {'location': {'description': 'Location to get weather information for', 'title': 'Location', 'type': 'string'}, 'unit': {'description': 'Unit of temperature', 'enum': ['celsius', 'fahrenheit'], 'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}


In [13]:
print(Calculator.definition)

{'type': 'function', 'function': {'name': 'Calculator', 'description': 'Evaluates a mathematical expression', 'parameters': {'properties': {'expression': {'description': 'The expression to be evaluated.', 'title': 'Expression', 'type': 'string'}}, 'required': ['expression'], 'type': 'object'}}}


In [14]:
print(EmailSearch.definition)

{'type': 'function', 'function': {'name': 'EmailSearch', 'description': 'Searches email addresses in a given text', 'parameters': {'properties': {'text': {'description': 'The target text in which emails are to be searched.', 'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'type': 'object'}}}


Let's create the list of tool definitions to be shared with the LLM.

In [15]:
tools = [WeatherInfo, Calculator, EmailSearch]

Let's create a tools dictionary so that once we know the name of the tool from LLM, we can call it with arguments returned from the LLM.

In [16]:
tools_dict = {tool.name: tool for tool in tools}

Pass tools to the user prompt with **tools** argument.

In [17]:
user_prompt = Prompt.user_prompt("What's the weather like in Boston?", tools=tools)
router.execute(user_prompt)

Executing Conversation with 1 step(s)


ToolResponse: 35°C

In [18]:
router.execute(Prompt.user_prompt("What is 7 *30 + 13?", tools=tools))

Executing Conversation with 1 step(s)


ToolResponse: 223

In [19]:
text = "This is a sample text with a@b.com and c@d.com addresses"
prompt_text = f"""Find whether there is an email address in the following text marked with triple backticks 
            ```{text}```
            """
router.execute(Prompt.user_prompt(prompt_text, tools=tools))

Executing Conversation with 1 step(s)


ToolResponse: ['a@b.com', 'c@d.com']